# Grover's algorithm
Grover's algorithm is a search algorithm to find out the data efficiently from random data.

The basic step of grover's algorithm is 

1. Making superposition  
2. Marking of the data you want to find  
3. Amplitude amplification  

Repeating the step2 and 3 properly, we can get the correct result.

# Install

In [ ]:
!pip install blueqat

## Marking
Here we see 2qubits grover's algorithm. Combination of 2 binary numbers are 00, 01, 10 and 11. Now we see the basic marking circuit to mark the state vector.

To do marking we prepare a diagonal matrix which is one element is -1 and others are 1. By using H gate, CZ gate and S gate we can realize it.


Let's see on blueqat.

Now we see the circuit one by one. We can check the unitary matrix of the circuit by running .run_with_sympy_unitary(). And we usually start from the diagonal matrix of CZ gate first and changing it.

In [1]:
from blueqat import Circuit

In [2]:
'''
#marking on 11

-------*-----
-------Z-----
'''

Circuit(2).cz[0,1].run_with_sympy_unitary()

Matrix([
[1, 0, 0,  0],
[0, 1, 0,  0],
[0, 0, 1,  0],
[0, 0, 0, -1]])

In [4]:
'''
#marking on 01
 
----S--*--S---
-------Z-------
'''

Circuit(2).s[0].cz[0,1].s[0].run_with_sympy_unitary()

Matrix([
[1,  0, 0, 0],
[0, -1, 0, 0],
[0,  0, 1, 0],
[0,  0, 0, 1]])

In [5]:
'''
#marking on 10
 
--------*------
----S--Z--S---
'''

Circuit(2).s[1].cz[0,1].s[1].run_with_sympy_unitary()

Matrix([
[1, 0,  0, 0],
[0, 1,  0, 0],
[0, 0, -1, 0],
[0, 0,  0, 1]])

In [6]:

'''
#00
 
----S--*--S--
----S--Z--S--
'''

Circuit(2).s[:].cz[0,1].s[:].run_with_sympy_unitary()

Matrix([
[1,  0,  0,  0],
[0, -1,  0,  0],
[0,  0, -1,  0],
[0,  0,  0, -1]])

The result is inversed on the sign. We can think about global phase that flips all the minus sign to plus and plus sign to minus.

## Amplitude amplification
Ampitude amplification is a circuit which is $\frac{1}{2}$ on diagonal and $-\frac{1}{2}$ on offdiagonal. Doing this we can get the marked state vector.

The amplitude amplification circuit on 2qubits is like this,

In [7]:
'''
--H-X-*-X-H--
--H-X-Z-X-H--
'''

Circuit(2).h[:].x[:].cz[0,1].x[:].h[:].run_with_sympy_unitary()

Matrix([
[ 1/2, -1/2, -1/2, -1/2],
[-1/2,  1/2, -1/2, -1/2],
[-1/2, -1/2,  1/2, -1/2],
[-1/2, -1/2, -1/2,  1/2]])

## Circuit
Now we have the total circuit. First we prepare the state vector as |+> applying H gate on every qubit.

In [46]:
from blueqat import Circuit

#Amplitude amplification
a = Circuit(2).h[:].x[:].cz[0,1].x[:].h[:].m[:]

'''
#00 Circuit
--H--S--*--S----H-X-*-X-H--
--H--S--Z--S----H-X-Z-X-H--
'''

(Circuit(2).h[:].s[:].cz[0,1].s[:] + a).run(shots=100)

Counter({'00': 100})

In [47]:
'''
#01 Circuit
--H-----*-------H-X-*-X-H--
--H--S--Z--S---H-X-Z-X-H--
'''

(Circuit(2).h[:].s[1].cz[0,1].s[1] + a).run(shots=100)

Counter({'01': 100})

In [48]:
'''
#10 Circuit
--H--S--*--S----H-X-*-X-H--
--H-----Z--------H-X-Z-X-H--
'''
(Circuit(2).h[:].s[0].cz[0,1].s[0] + a).run(shots=100)

Counter({'10': 100})

In [49]:
'''
#11 Circuit
--H-----*-------H-X-*-X-H--
--H-----Z-------H-X-Z-X-H--
'''
(Circuit(2).h[:].cz[0,1] + a).run(shots=100)

Counter({'11': 100})

And we all succeded to get the result.

# Explanation of the algorithm

## Oracle
Let's start with the oracle that appears in this algorithm.
In simple terms, it is a function that returns 0 or 1 for a given input.  
As an example, let's consider the situation of buying something. Let's say that the number of items you can buy at the butcher shop is 1 and the number of items you cannot buy is 0.

Then you will have the following situation.

$$
f(x) = \begin{cases}
   1\ \  (\text{You can buy $x$ at the butcher shop})\\
    0\ \  (\text{You can NOT buy $x$ at the butcher shop})
  \end{cases}
$$

Using this, we can't buy PCs at the butcher shop, but we can buy beef.  
So,

$$
f(\text{PC}) = 0,\ \ \ \ f(\text{beef}) = 1
$$

The function that separates 0 and 1 in this way is called an oracle.

In this case, we will consider an Oracle that returns 1 fir only one case.  
In other words, if the input of $f$ is 00...00 to 11...11, only one of them returns 1, and all the others return 0.


$$
f(x) = \begin{cases}
   1\ \  (x= \omega)\\
    0\ \  (x\neq\omega)
  \end{cases}
$$

Here $f$ returns 1 if $x = \omega$.  

### Amplitude amplification
The word "amplitude" came up.  
The coefficients for each state are called **amplitude** and the probability is called **probability amplitude**.

Let's look at the following equation.

$$
H \lvert 0 \rangle \otimes H \lvert 0 \rangle = \frac{1}{2} (\lvert 00 \rangle + \lvert 01 \rangle + \lvert10 \rangle + \lvert 11 \rangle) \xrightarrow{\text{Amplitude amplification}} \lvert 00 \rangle
$$

In this case, the probability amplitudes on the left side of the arrow are all $1/4$, while the state on the right side has probability amplitude $1$ for $00$.

Amplitude amplification, as the word implies, is an algorithm that amplifies this probability amplitude as shown above.

The following figure illustrates the calculations.

<img width="30%" src="https://upload.wikimedia.org/wikipedia/commons/1/16/Grovers_algorithm_geometry.png">

参考: https://en.wikipedia.org/wiki/Grover%27s_algorithm

First, let's explain the symbols.
$s$ is the superposition of all bits.

$$
\lvert s \rangle = \otimes^n  H  \lvert 0 \rangle =\frac{1}{\sqrt{2^n}}\sum^{2^n}_{x\in \{0, 1\}^n}\lvert x \rangle
$$

Of these, $\omega$ is the state we want to amplify.

$$
\lvert \omega \rangle = \frac{1}{\sqrt{2^n}} \lvert 00...010...00\rangle
$$

Suppose that the $x$-th number contains a 1.

$s'$ is a vector of $s$ minus $\omega$.

$$
\lvert s' \rangle = \lvert s \rangle - \lvert \omega \rangle = \lvert \omega^{\perp} \rangle
$$

You can see that it is perpendicular to $\omega$.

Next, consider the unit circle consisting of $s'$ and $\omega$ as shown in the figure above.  
However, since the lengths of $s'$ and $\omega$ are different, we normalize them.

$$
\tilde{\lvert s' \rangle} = \sqrt{\frac{2^n}{2^n-1}} \lvert s' \rangle 
$$
$$
\tilde{\lvert \omega \rangle} = \sqrt{2^n} \lvert \omega \rangle
$$

From this, the vector $\psi$ on the unit circle can be expressed as follows

$$
\lvert \psi \rangle = \cos\phi \tilde{\lvert s' \rangle} + \sin\phi \tilde{\lvert \omega \rangle}
$$

$U_{\omega}$ is a matrix that inverts $\psi$ on the $s'$ axis.
In other words, rotate $\psi$ by $-\phi$.

$$
U_{\omega} \lvert \psi \rangle = \cos(-\phi) \tilde{\lvert s' \rangle} + \sin(-\phi) \tilde{\lvert \omega \rangle} = \cos(\phi) \tilde{\lvert s' \rangle} - \sin(\phi) \tilde{\lvert \omega \rangle}
$$

$U_s$ is a matrix that inverts $\psi$ around $s$.

$$
U_s \lvert \psi \rangle = \cos \bigg\{ \frac{\theta}{2} + \big(\frac{\theta}{2} - \phi \big) \bigg\} \tilde{\lvert s' \rangle} + \sin \bigg\{ \frac{\theta}{2} + \big(\frac{\theta}{2} - \phi \big) \bigg\}\tilde{\lvert \omega \rangle} = \cos(\theta - \phi)\tilde{\lvert s' \rangle} + \sin(\theta - \phi) \tilde{\lvert \omega \rangle}
$$

#### Overview
The algorithm is outlined below.

1. Invert $s$ around $s'$ using $U_{\omega}$.

2. Invert $U_{\omega}s$ around $s$ using $U_s$.

We will explain the above process in detail.

#### 1. Invert around $s'$

Using $\theta$ as shown in the figure above, we describe $s$ as following.

$$
\lvert s \rangle = \cos\bigl(\frac{\theta}{2}\bigr) \tilde{\lvert s' \rangle} - \sin\bigl(\frac{\theta}{2}\bigr) \tilde{\lvert \omega \rangle}
$$

As a same time,

$$
\cos\bigl( \frac{\theta}{2} \bigr) = \sqrt{\frac{2^n-1}{2^n}},\ \ \ \  \sin\bigl( \frac{\theta}{2} \bigr) = \sqrt{\frac{1}{2^n}}
$$

Using $U_{\omega}$, fold $s$ around $s'$.  
From the above figure, described as follows

$$
U_{\omega} \lvert s \rangle = \cos\bigl(-\frac{\theta}{2}\bigr)\tilde{\lvert s' \rangle} + \sin\bigl(-\frac{\theta}{2}\bigr)\tilde{\lvert \omega \rangle} = \cos\bigl(\frac{\theta}{2}\bigr)\tilde{\lvert s' \rangle} - \sin\bigl(\frac{\theta}{2}\bigr)\tilde{\lvert \omega \rangle}
$$

Since we are only working on $\omega$ for this operation, we can see that $U_{\omega}$ represents Oracle as described above.

#### 2. Invert around $s$
Using $U_s$, fold $U_{\omega}s$ around $s$.

$$
U_s U_{\omega} \lvert s\rangle = U_s\biggl( \cos\bigl(-\frac{\theta}{2}\bigr)\tilde{\lvert s' \rangle} + \sin\bigl(-\frac{\theta}{2}\bigr)\tilde{\lvert \omega \rangle}  \biggr)
$$

Since we need to rotate $2\theta$ here,

$$
U_s U_{\omega} \lvert s\rangle =  \cos\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert s' \rangle} + \sin\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert \omega \rangle}
$$

Specifically, $\cos$ and $\sin$ are obtained from the additive theorem.

$$
\cos \frac{3}{2}\theta = \bigl( 1-\frac{4}{2^n} \bigr) \sqrt{\frac{2^n-1}{2^n}},\ \ \ \ \sin \frac{3}{2}\theta = \bigl( 3-\frac{4}{2^n} \bigr) \sqrt{\frac{1}{2^n}}
$$

Thus, using $s'$, $\omega$, we get

$$
U_s U_{\omega} \lvert s\rangle =  \bigl(1 - \frac{4}{2^n}\bigr)\lvert s' \rangle + \bigl(3 - \frac{4}{2^n}\bigr)\lvert \omega \rangle
$$

This operation makes $\omega$ of the $2^n$ amplitudes about three times larger than the others.  
We were able to amplify the amplitude by the above.

## Grover's algorithm
This algorithm can find $\omega$ given the Oracle above.

The content is to repeatedly amplify the amplitude to bring $s$ closer to $\omega$.

From the above equation, if we perform amplitude amplification once, we get

$$
\cos\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert s' \rangle} + \sin\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert \omega \rangle}
$$

Therefore, if we perform amplitude amplification $n$ times, we get

$$
\cos\bigl(\frac{1}{2}\theta + n\theta \bigr)\tilde{\lvert s' \rangle} + \sin\bigl(\frac{1}{2}\theta + n\theta\bigr)\tilde{\lvert \omega \rangle} = \cos\biggl\{\bigl(\frac{1}{2} + n \bigr)\theta \biggr\} \tilde{\lvert s' \rangle} + \sin\biggl\{\bigl(\frac{1}{2} + n \bigr)\theta \biggr\}\tilde{\lvert \omega \rangle}
$$

As the amplitude is amplified, the initial state $s$ in the above figure becomes closer and closer to $\omega$.
In other words, the probability amplitude of $\omega$ goes to 1.
However, if you keep repeating the process, it will just keep rotating, so you need to think about the number of times you want to repeat it.

Since we want to set the probability amplitude of $\omega$ to 1,

$$
\sin^2\biggl\{\bigl(\frac{1}{2} + k \bigr)\theta \biggr\} < 1 \xrightarrow{\sin >0}\bigl(\frac{1}{2} + k \bigr)\theta < \frac{\pi}{2} \Leftrightarrow k<\frac{\pi}{2\theta} - \frac{1}{2}
$$

From the above, the number of times the amplitude is amplified is determined and $\omega$ can be obtained.

## Supplement (Application to quantum programming)
In quantum programming, amplitude amplification is considered as follows.

If we separate $s = s' + \omega$ from the definition of $s$, then $U_{\omega}$ is simply

$$
U_{\omega} (\lvert s' \rangle + \lvert \omega \rangle) = \lvert s' \rangle - \lvert \omega \rangle
$$

This means that we can use gates that change sign only for certain states, such as Z and CZ gates.

Thinking geometrically from the above figure, $U_s$ is described as follows.

$$
\begin{align}
U_s U_{\omega} \lvert s \rangle &= 2(\langle s \lvert U_{\omega} \rvert s \rangle \rvert s \rangle - U_{\omega}\lvert s \rangle) + U_{\omega}\lvert s \rangle \\
&= 2\lvert s \rangle \langle s \lvert U_{\omega} \rvert s \rangle - U_{\omega}\lvert s \rangle \\
&= (2\lvert s\rangle \langle s\rvert - I) U_{\omega}\lvert s \rangle
\end{align}
$$

So $U_s = 2\lvert s\rangle \langle s\rvert - I$.

Furthermore, $U_s$ can be decomposed as follows.

$$
2\lvert s\rangle \langle s\rvert - I = 2H^{\otimes n}\lvert 0^n\rangle \langle 0^n\rvert H^{\otimes n} - I = H^{\otimes n} (2\lvert 0^n\rangle \langle 0^n\rvert - I) H^{\otimes n}\ \ \ \ (\lvert 0^n \rangle = \lvert 00\cdots 00 \rangle)
$$

Here, we can write $2\lvert 0^n\rangle \langle 0^n\rvert - I$ as

$$
2\lvert 0^n\rangle \langle 0^n\rvert - I = 
    \begin{pmatrix}
      -1 & 0 & 0 & \ldots & 0 & 0 \\
      0 & 1 & 0 & \ldots & 0 & 0 \\
      0 & 0 & 1 & \ldots & 0 & 0 \\
      \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
      0 & 0 & 0 & \ldots & 1 & 0 \\
      0 & 0 & 0 & \ldots & 0 & 1
    \end{pmatrix}
$$

We can describe it as following.
$$
XZX =
    \begin{pmatrix}
      -1 & 0 \\
      0 & 1
    \end{pmatrix}
$$

$$
2\lvert 0^n\rangle \langle 0^n\rvert - I = X^{\otimes n}C^n ZX^{\otimes n}
$$


From the above, we were able to rewrite Grover's algorithm with simple gate sets.